In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# Pre-preprocessing
 - (пришлось отредактировать 14 строку 3 листа, там странный формат id)

In [2]:
for i in range(1, 4):
    pd.read_excel("Taxonomy examples (edited).xlsx", sheet_name=i - 1, header=None).to_csv(f"tree{i}.csv", index=False)

In [3]:
def strip_csv(file_path):
    """
    Auxiliary function for specific dataset.
    """
    with open(file_path, 'r') as file:
        stripped_lines = [line[1:-2] for line in file]

    with open(file_path, 'w') as file:
        file.write('\n'.join(stripped_lines[1:]))
        
for file in ['tree1.csv', 'tree2.csv', 'tree3.csv']:
    strip_csv(file)

In [4]:
naming = ["id"]+list(range(10))
df1 = pd.read_csv('tree1.csv', names=naming)
df2 = pd.read_csv('tree2.csv', names=naming)
df3 = pd.read_csv('tree3.csv', names=naming)
df1.head()

,id,0,1,2,3,4,5,6,7,8,9
0,3.2.1.,NaN,NaN,Data mining,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.2.1.1.,NaN,NaN,NaN,Data cleaning,NaN,NaN,NaN,NaN,NaN,NaN
2,3.2.1.2.,NaN,NaN,NaN,Collaborative filtering,NaN,NaN,NaN,NaN,NaN,NaN
3,3.2.1.2.1*,NaN,NaN,NaN,NaN,Item-based,NaN,NaN,NaN,NaN,NaN
4,3.2.1.2.2*,NaN,NaN,NaN,NaN,Scalable,NaN,NaN,NaN,NaN,NaN


In [5]:
reg = r"[*.]+(?!.*[*.])" # Matches tracing symbol (either ., * or **), did not find interpretation of it
df1["id"] = df1["id"].str.replace(reg, "")
df2["id"] = df2["id"].str.replace(reg, "")
df3["id"] = df3["id"].str.replace(reg, "")

# DF to directed graph

In [6]:
# !pip install networkx

In [7]:
import networkx as nx
import re

def df_to_graph(data):
    """
    Auxiliary function to transform dataframe into nx.DiGraph
    """
    G = nx.DiGraph()
    for r in data.iterrows():
        index = str(r[1].id)
        name = r[1][~r[1].isna()].iloc[1]
        G.add_node(name)
        
        parent_index = index[:-len(re.findall(r"\d+(?!.*\d+)", index)[0])-1]
        parent = data[data.id == parent_index]
        if parent.shape[0] == 0:
            continue

            
        parent = parent.iloc[0]
        parent_name = parent[~parent.isna()].values[1]  
        
        if parent_name in G:
            G.add_edge(parent_name, name)
            
    return G
    
tree1 = df_to_graph(df1)
tree2 = df_to_graph(df2)
tree3 = df_to_graph(df3)

In [8]:
# tree1.nodes

In [9]:
# tree2.edges

In [10]:
# def get_leaves(graph, node): # Redundant function
#     leaves = []
#     for u, children in nx.dfs_successors(graph, node).items():
#         for v in children:
#             if graph.out_degree(v) == 0:
#                 leaves.append(v)
#     return leaves

# get_leaves(tree2, 'Sequence mining')

# Least Common Ancestor mapping

In [11]:
def least_common_ancestor(graph: nx.DiGraph, nodes: list):
    """
    Least common ancestor: the 'deepest' node in graph, containing all nodes as successors.
    """
    ancestors = [set(nx.ancestors(graph, node)) for node in nodes]
    common_ancestors = set.intersection(*ancestors)
    
    return min(common_ancestors, key=lambda x: len(nx.shortest_path(graph, x, nodes[0])) - 1)

least_common_ancestor(tree2, ['Rule and pattern discovery', 'Trajectory clustering', 'Types of association rules'])

'Data mining'

In [12]:
def map_tree(treeG: nx.DiGraph, treeS: nx.DiGraph):
    """
    Least common ancestor mapping. (Assumes presence of all nodes from treeG in treeS.)
    
    Maps every node X from treeG to least common ancestor node of all successors of X from treeG in treeS (unless it does not have any, in which case maps it to itself).
    """
    mappings = []
    for node in treeG.nodes:
        if treeG.in_degree(node) != 0 and treeG.out_degree(node) != 0:
            successors = list(nx.dfs_successors(treeG, node).values())[0]
            lca = least_common_ancestor(treeS, successors)

            mappings += [(node, lca)]
            # print(f"{node} ---> {lca}")
        else:
            mappings += [(node, node)]
            # print(f"{node} ---> {node}")
    
    mappings = {key: value for (key, value) in mappings}
    return mappings
            
        
print(map_tree(tree1, tree3))

{'Data mining': 'Data mining', 'Data cleaning': 'Data cleaning', 'Collaborative filtering': 'Collaborative filtering', 'Item-based': 'Item-based', 'Scalable': 'Scalable', 'Association rules': 'Association rules', 'Types of association rules': 'Types of association rules', 'Interestingness': 'Interestingness', 'Parallel computation': 'Parallel computation', 'Clustering': 'Clustering', 'Massive data clustering': 'Massive data clustering', 'Consensus clustering': 'Consensus clustering', 'Fuzzy clustering': 'Fuzzy clustering', 'Additive clustering': 'Additive clustering', 'Feature weight clustering': 'Feature weight clustering', 'Conceptual clustering': 'Conceptual clustering', 'Biclustering': 'Biclustering', 'Nearest-neighbor search': 'Nearest-neighbor search', 'Data stream mining': 'Data stream mining', 'Graph mining': 'Graph mining', 'Graph partitioning': 'Graph partitioning', 'Frequent graph mining': 'Frequent graph mining', 'Graph based conceptual clustering': 'Graph based conceptual 

# Duplications

In [13]:
def get_duplications(treeG: nx.DiGraph, treeS: nx.DiGraph, mappings=None):
    """
    Function for determining the duplications in two mapped trees.
    Partial duplication - node in treeG, one of whose children's mapping is the same as its own.
    Full duplication - node in treeG, all of whose childrens' mappings are the same as its own.
    
    Returns two lists: full duplications and partial duplications.
    """
    partial = []                                           # Analogy for one-side duplications
    full = []                                              # Analogy for two-side duplications
    
    if not mappings:
        mappings = map_tree(treeG, treeS)
        
    
    for m in mappings:
        own = mappings[m]
        children = [mappings[s] for s in treeG.successors(m)]
        
        is_full = bool(children)                           # So that leaves are not considered two-side duplications
        is_partial = False
        for c in children:
            if c != own:
                is_full = False
            else:
                is_partial = True
        
        if is_full:
            full.append(m)
        elif is_partial:
            partial.append(m)
        
    return full, partial

# get_duplications(tree1, tree3)

In [14]:
gene_tree = df_to_graph(pd.read_csv('genetree.csv', names=["id"]+list(range(5))))
species_tree = df_to_graph(pd.read_csv('speciestree.csv', names=["id"]+list(range(5))))

In [15]:
map_tree(gene_tree, species_tree)

{'a': 'a',
 'b': 'a',
 'c': 'c',
 'd': 'c',
 'one': 'one',
 'two': 'two',
 'three': 'three',
 'e': 'g',
 'four': 'four',
 'five': 'five',
 'f': 'a',
 'six': 'six',
 'g': 'e',
 'seven': 'seven',
 'eight': 'eight'}

In [16]:
print(get_duplications(gene_tree, species_tree))

(['a'], ['c'])


# Intermediate nodes

In [17]:
def get_intermediates(treeG: nx.DiGraph, treeS: nx.DiGraph, mappings=None, root=None):
    """
    Function to get the number of intermediate nodes in two mapped trees.
    For node g in treeG, nodes in treeS are called intermediate if they "lie inbetween" g's mapping and g's parent's mapping.
    
    Returns integer - number of intermediate nodes.
    """
    intermediates = 0
    
    if not mappings:
        mappings = map_tree(treeG, treeS)
        
    if not root:
        root = list(mappings.keys())[0]                     # Костыль, лучше заранее сохранять корень (или переехать с networkx на ete3)
        
    for node in treeG.nodes:
        parent = next(treeG.predecessors(node), None)
        if not parent:
            continue
            
        kek = max(nx.shortest_path_length(treeS, mappings[parent], mappings[node]) - 1, 0)
        intermediates += kek
        
        
    return intermediates
    
get_intermediates(gene_tree, species_tree)

7

# Cost function

In [18]:
def tree_cost(treeG: nx.DiGraph, treeS: nx.DiGraph, mappings=None, root=None):
    partial_duplications = len(get_duplications(treeG, treeS, mappings)[1])
    intermediates = get_intermediates(treeG, treeS, mappings, root)
    
    return partial_duplications + intermediates

tree_cost(gene_tree, species_tree)

8

# Visualization

In [19]:
import ete3

In [20]:
def convert_to_ete(initial: nx.DiGraph):
    """
    Auxiliary function that converts nx.DiGraph to an ete3 Tree in order to simplify visualization.
    """
    subtrees = {node:ete3.Tree(name=node) for node in initial.nodes()}
    [*map(lambda edge:subtrees[edge[0]].add_child(subtrees[edge[1]]), initial.edges())]
    
    return subtrees[list(initial.nodes)[0]]

# print(convert_to_ete(tree1).get_ascii())

In [21]:
print(convert_to_ete(gene_tree).get_ascii())


            /-one
         /d|
      /c|   \-two
     |  |
   /b|   \-three
  |  |
  |  |   /-four
-a|   \e|
  |      \-five
  |
  |   /-six
   \f|
     |   /-seven
      \g|
         \-eight


In [22]:
print(convert_to_ete(species_tree).get_ascii())


            /-one
         /d|
      /c|   \-three
     |  |
   /b|   \-two
  |  |
  |   \-six
  |
-a|         /-four
  |      /g|
  |   /f|   \-five
  |  |  |
   \e|   \-seven
     |
      \-eight


# Random tree generation
<font size=3>Auxiliary functions to test algorithm above</font>

In [42]:
import random

def create_random_tree(n: int):
    """
    Creates a random tree as a nx.DiGraph with n nodes.
    """
    G = nx.DiGraph()
    G.add_node(1)

    for i in range(2, n+1):
        parent = random.choice(list(G.nodes()))

        G.add_edge(parent, i)

    return G


print(convert_to_ete(create_random_tree(15)).get_ascii())


   /2 /-11
  |
  |         /-13
-1|   /4 /10
  |  |      \-14
  |  |
   \3|-5 /-12
     |
     |   /7 /8 /-15
      \6|
         \-9


In [43]:
# def apply_random_permutations(G: nx.DiGraph, num_permutations: int):
#     """
#     Applies num_permutations random permutations to the tree G.
#     """
#     for i in range(num_permutations):
#         node = random.choice(list(G.nodes()))

#         # Apply a random permutation to the children of the node
#         children = list(G.successors(node))
#         random.shuffle(children)
#         G.remove_edges_from([(node, child) for child in children])
#         G.add_edges_from([(node, child) for child in children])

#     return G

# random_tree = create_random_tree(15)
# print(convert_to_ete(random_tree).get_ascii())
# print(convert_to_ete(apply_random_permutations(random_tree, 4)).get_ascii())

# Leaves' permutations

In [44]:
def get_leaves(graph: nx.DiGraph):
    """
    Returns the list of leaves in a tree
    """
    leaves = [node for node in graph.nodes() if graph.out_degree(node) == 0]
    return leaves

get_leaves(gene_tree)

['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight']

In [46]:
def swap_nodes(graph, rate, nodes):
    num_swaps = int(len(nodes) * rate)
    nodes_to_swap = random.sample(nodes, num_swaps)
    new_graph = graph.copy()
    for node in nodes_to_swap:
        other_node = random.choice([other for other in nodes if other != node])
        parent = list(new_graph.predecessors(node))[0]
        other_parent = list(new_graph.predecessors(other_node))[0]
        new_graph.remove_edge(parent, node)
        new_graph.remove_edge(other_parent, other_node)
        new_graph.add_edge(other_parent, node)
        new_graph.add_edge(parent, other_node)
    return new_graph

print(convert_to_ete(gene_tree).get_ascii())
print(convert_to_ete(swap_nodes(gene_tree, 0.5, get_leaves(gene_tree))).get_ascii())


            /-one
         /d|
      /c|   \-two
     |  |
   /b|   \-three
  |  |
  |  |   /-four
-a|   \e|
  |      \-five
  |
  |   /-six
   \f|
     |   /-seven
      \g|
         \-eight

            /-one
         /d|
      /c|   \-four
     |  |
   /b|   \-three
  |  |
  |  |   /-two
  |   \e|
-a|      \-six
  |
  |      /-seven
  |   /g|
   \f|   \-eight
     |
      \-five
